# Análisis exploratorio de incidencia delictiva en México

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv('data/delitos.csv')

#### Ejercicio 1: Elige 3 estados de la república y grafica una serie de tiempo de la frecuencia abosluta de homicidios dolosos de enero 2015 a julio 2019 en estas tres entidades

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Leer datos
delitos = pd.read_csv('data/ejercicios_pandas_delitos.csv', encoding='iso-8859-1')

# Convertir de ancho a largo
meses = ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio',
         'Agosto','Septiembre','Octubre','Noviembre','Diciembre']

delitos_long = delitos.melt(
    id_vars=['A�o','Clave_Ent','Entidad','Tipo de delito','Subtipo de delito'],
    value_vars=meses,
    var_name='mes',
    value_name='frecuencia'
)

# Renombrar columnas
delitos_long = delitos_long.rename(columns={'A�o':'anio','Clave_Ent':'clave_entidad','Entidad':'entidad','Tipo de delito':'tipo_de_delito'})

# Filtrar solo Homicidio doloso
hom_doloso = delitos_long[delitos_long['Subtipo de delito']=='Homicidio doloso']

# Filtrar CDMX, Nuevo León y Jalisco
estados = ['Ciudad de México','Nuevo León','Jalisco']
hom_doloso_estados = hom_doloso[hom_doloso['entidad'].isin(estados)]

# Agrupar por año y mes
serie_tiempo = hom_doloso_estados.groupby(['anio','mes','entidad'])['frecuencia'].sum().reset_index()

# Graficar
plt.figure(figsize=(12,6))
for estado in estados:
    df = serie_tiempo[serie_tiempo['entidad']==estado]
    plt.plot(df['anio'].astype(str) + '-' + df['mes'], df['frecuencia'], label=estado)

plt.xticks(rotation=90)
plt.ylabel('Homicidios dolosos')
plt.title('Serie de tiempo mensual de homicidios dolosos (CDMX, NL, Jalisco)')
plt.legend()
plt.show()


#### Ejercicio 2: Contetas las siguientes  preguntas:
1. ¿Cuántos homicidios dolosos hubo en Colima en el 2018?
2. ¿Cuantos robos de vehículo automotor ha habido en el 2019?
3. Obten la suma de homicidos dolosos y feminidios en toda la República Mexicana en cada año.
4. ¿En qué mes y en qué municipio ha ocurrido el mayor número de feminicidios?
5. ¿En qué año y en qué estado ha ocurrido el mayor número de feminicidios?

In [ ]:
# Leer población
pobs = pd.read_csv('data/poblacion_entidades_2015.csv', encoding='iso-8859-1', sep=";")
pobs = pobs[['Cve_Entidad', 'Entidad', 'Poblacion']]
pobs = pobs.rename(columns={'Cve_Entidad':'clave_entidad', 'Entidad':'entidad', 'Poblacion':'poblacion'})

# Pregunta 1: homicidios dolosos en Colima en 2018
colima_2018 = hom_doloso[(hom_doloso['entidad']=='Colima') & (hom_doloso['anio']==2018)]
print("Homicidios dolosos en Colima 2018:", colima_2018['frecuencia'].sum())

# Pregunta 2: robos de vehiculo automotor en 2019
robo_vehiculo = delitos_long[(delitos_long['Subtipo de delito']=='Robo de vehículo automotor') & (delitos_long['anio']==2019)]
print("Robos de vehículo en 2019:", robo_vehiculo['frecuencia'].sum())

# Pregunta 3: suma de homicidios dolosos y feminicidios en toda la República por año
feminicidios = delitos_long[delitos_long['Subtipo de delito']=='Feminicidio']
total_delitos = pd.concat([hom_doloso, feminicidios])
suma_anual = total_delitos.groupby('anio')['frecuencia'].sum()
print("Suma anual de homicidios dolosos y feminicidios:")
print(suma_anual)

# Pregunta 4: mes y municipio con mayor feminicidio
mayor_fem = feminicidios.loc[feminicidios['frecuencia'].idxmax()]
print("Mes con mayor feminicidio:", mayor_fem['mes'], "en entidad:", mayor_fem['entidad'])

# Pregunta 5: año y estado con mayor feminicidio
mayor_fem_ano = feminicidios.groupby(['anio','entidad'])['frecuencia'].sum().idxmax()
print("Año y estado con mayor feminicidio:", mayor_fem_ano)


#### Ejercicio 3: Haz una gráfica de pastel de tipos de delito. Deberás crear una gráfica para cada año. Utilzia la función subplots de matplotlib

In [ ]:
import matplotlib.pyplot as plt

# Sumar por tipo de delito y año
delitos_anual = delitos_long.groupby(['anio','tipo_de_delito'])['frecuencia'].sum().reset_index()

# Lista de años
anios = delitos_anual['anio'].unique()
n_anios = len(anios)

# Crear subplots
fig, axes = plt.subplots(1, n_anios, figsize=(5*n_anios,5))

for i, anio in enumerate(anios):
    ax = axes[i] if n_anios > 1 else axes
    datos = delitos_anual[delitos_anual['anio']==anio]
    ax.pie(datos['frecuencia'], labels=datos['tipo_de_delito'], autopct='%1.1f%%')
    ax.set_title(f'Tipo de delito {anio}')

plt.show()


---
#### Calcula la tasa por 100,000 habitantes
##### Tasa por 100,000 habitantes
Mostrar el total de delitos en una entidad no nos sirve de mucho. Es mucho más útil calcular la tasa de incidencia delictiva por cada 100,000 habitantes

$$
tasa = \frac{delitos\space totales}{población} \times 100,000
$$

Esta tasa la podemos anualizar multiplicándola por un factor de 12
$$
tasa\space anualizada = tasa \times 12
$$

Población por entidad federativa según [la encuesta intercensal 2015](https://www.inegi.org.mx/programas/intercensal/2015/)

No tienes que descargar nada. Ya están los datos en la carpeta data

In [ ]:
pobs = pd.read_csv('data/poblacion_entidades_2015.csv', encoding='iso-8859-1', sep=";")
pobs = pobs[['Cve_Entidad', 'Entidad', 'Poblacion']]
pobs = pobs.rename(columns={'Cve_Entidad':'clave_entidad', 'Entidad':'entidad', 'Poblacion':'poblacion'})
pobs.head()

In [ ]:
# Unir con población
delitos_long = delitos_long.merge(pobs, on=['clave_entidad','entidad'], how='left')

# Calcular tasa mensual por 100,000 habitantes
delitos_long['tasa_100mil_mensual'] = (delitos_long['frecuencia'] / delitos_long['poblacion']) * 100000

delitos_long.head()




In [ ]:
# Multiplicar por 12
delitos_long['tasa_100mil_anual'] = delitos_long['tasa_100mil_mensual'] * 12
delitos_long.head()
